In [4]:
import numpy as np
import pandas as pd
import chess
import chess.pgn
from ChessFunctions import get_encoded_board

In [2]:
f = open('lichess_elite_2020-06.pgn')

In [3]:
my_list = []
Y_labels = []
for game in range(4000):
# while True:
    game = chess.pgn.read_game(f)
    if game is None:
        break  # end of file
    my_list.append(game)

In [15]:
def flatten_board(x):
    return(str(x).replace(' ','').replace('\n',''))

In [38]:
def get_encoded_board(x):
    num_squares = range(64)

    x = flatten_board(x)
    #Black Player Feature Vectors
    b_rook  = [1 if x[i]=='r' else 0 for i in num_squares]
    b_night = [1 if x[i]=='n' else 0 for i in num_squares]
    b_bish  = [1 if x[i]=='b' else 0 for i in num_squares]
    b_queen = [1 if x[i]=='q' else 0 for i in num_squares]
    b_king  = [1 if x[i]=='k' else 0 for i in num_squares]
    b_pawn  = [1 if x[i]=='p' else 0 for i in num_squares]
    b_value = (sum(b_rook)*5 + sum(b_night)*3 + sum(b_bish)*3
           + sum(b_queen)*9 + sum(b_pawn)*1
          )
    
    #White Player Feature Vectors
    w_rook  = [1 if x[i]=='R' else 0 for i in num_squares]
    w_night = [1 if x[i]=='N' else 0 for i in num_squares]
    w_bish  = [1 if x[i]=='B' else 0 for i in num_squares]
    w_queen = [1 if x[i]=='Q' else 0 for i in num_squares]
    w_king  = [1 if x[i]=='K' else 0 for i in num_squares]
    w_pawn  = [1 if x[i]=='P' else 0 for i in num_squares]
    w_value = (sum(w_rook)*5 + sum(w_night)*3 + sum(w_bish)*3
           + sum(w_queen)*9 + sum(w_pawn)*1
          )

    feature_list = np.array(b_rook+b_night+b_bish+b_queen
                +b_king+b_pawn+w_rook+w_night+w_bish
                +w_queen+w_king+w_pawn).reshape([1,768])
    
    feature_list = np.append(feature_list,[w_value,b_value])
    
    return(feature_list)

In [39]:
game_state_dictionary = {}
Y_labels = []
i = 0
for game in my_list:
    board = game.board()
    board_state = get_encoded_board(board)
    # Win
    if game.headers['Result'] == "1-0":
        Y_labels.append("w")
    # Lose
    elif game.headers['Result'] == "0-1":
        Y_labels.append('l')
    # Draw
    else:
        Y_labels.append('d')
    for move in game.mainline_moves():
        board.push(move)
        board_state = np.row_stack((board_state,get_encoded_board(board)))
        # Win
        if game.headers['Result'] == "1-0":
            Y_labels.append('w')
        # Lose
        elif game.headers['Result'] == "0-1":
            Y_labels.append('l')
        # Draw
        else:
            Y_labels.append('d')
    game_state_dictionary[i] = board_state
    i = i+1

In [40]:
len(Y_labels)

332834

In [41]:
X = np.row_stack([game_state_dictionary[x] for x in sorted(game_state_dictionary)])
X.shape

(332834, 770)

In [46]:
%%time
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y_labels, stratify=Y_labels,random_state=1)
clf = MLPClassifier(hidden_layer_sizes=(400,300,200),random_state=1
                    ,early_stopping=True,tol=.00001,max_iter=300).fit(X_train, y_train)
clf.predict_proba(X_test)

CPU times: user 2h 19min 38s, sys: 14min 12s, total: 2h 33min 50s
Wall time: 55min 17s


array([[4.31825056e-01, 1.45686006e-01, 4.22488938e-01],
       [7.38428375e-02, 3.11146975e-08, 9.26157131e-01],
       [3.25987168e-06, 9.95547477e-01, 4.44926291e-03],
       ...,
       [5.69838381e-10, 9.99957370e-01, 4.26292903e-05],
       [1.00000000e+00, 7.49547400e-13, 2.48557299e-15],
       [9.60428488e-09, 8.08378714e-01, 1.91621276e-01]])

In [47]:
clf.score(X_test, y_test)

0.8914660673725198

In [48]:
# clf.predict(X_test)

In [49]:
from joblib import dump, load
dump(clf, 'filename.joblib')  

['filename.joblib']

In [50]:
clf = load('filename.joblib') 